In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

pd.options.mode.chained_assignment = None  # default='warn'
warnings.simplefilter(action='ignore', category=FutureWarning)

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
data_test = pd.read_csv("../input/pubg-finish-placement-prediction/test_V2.csv")
data_train = pd.read_csv("../input/pubg-finish-placement-prediction/train_V2.csv")


**Eda**

In [ ]:
print(f"Розмір даних train {data_train.shape}")
print(f"Розмір даних test {data_test.shape}")

In [ ]:
data_train.head ()

In [ ]:
del data_train['Id']
del data_train['groupId']
del data_train['matchId']
del data_test['groupId']
del data_test['matchId']

In [ ]:
y_train = data_train['winPlacePerc']

In [ ]:
print(data_train.isnull().sum().sort_values(ascending=False)/len(data_train)*100)

In [ ]:
print(data_test.isnull().sum().sort_values(ascending=False)/len(data_test)*100)

In [ ]:
data_train = data_train[data_train['winPlacePerc'].notna()]

In [ ]:
data_train.dtypes

In [ ]:
data_train.describe()

In [ ]:
data_test.describe()

In [ ]:
plt.figure(figsize=(22,12))
sns.histplot(y_train)

In [ ]:
plt.figure(figsize=(22,12))
sns.boxplot(x=data_train['kills'])

In [ ]:
print(data_train['kills'].value_counts())

In [ ]:
plt.figure(figsize=(22,12))
data_train= data_train[data_train['kills']<12]
sns.countplot(data_train['kills'])

In [ ]:
print(data_train['assists'].value_counts())
data_train = data_train[data_train['assists']<7]

In [ ]:
plt.figure(figsize=(22,12))
sns.boxplot(x=data_train['assists'])

In [ ]:
plt.figure(figsize=(22,12))
sns.countplot(data_train[data_train['assists']<6]['assists'])

In [ ]:
sns.boxplot(data_train['heals'])

In [ ]:
data_train = data_train[data_train['heals']<51]

In [ ]:
data_train.hist(figsize=(16, 20), color = 'g', bins=30, xlabelsize=8, ylabelsize=8)

In [ ]:
data_test.hist(figsize=(16, 20), color = 'g', bins=30, xlabelsize=8, ylabelsize=8)

In [ ]:
plt.subplots(figsize=(24, 18))
sns.heatmap(data_train.corr(), annot=True,)

***feature engineering***

In [ ]:
data_train = pd.read_csv("../input/pubg-finish-placement-prediction/train_V2.csv")
data_test = pd.read_csv("../input/pubg-finish-placement-prediction/test_V2.csv")

In [ ]:
data_train = data_train[data_train['winPlacePerc'].notna()]

In [ ]:
data_train= data_train[data_train['kills']<12]

In [ ]:
data_train= data_train[data_train['assists']<7]

In [ ]:
data_train= data_train[data_train['heals']<51]

In [ ]:
data_train.head()

In [ ]:
data_train['playersJoined'] = data_train.groupby('matchId')['matchId'].transform('count')
data = data_train.copy()
data = data[data['playersJoined']>49]
plt.figure(figsize=(15,10))
sns.countplot(data['playersJoined'])
plt.title("Players Joined",fontsize=15)
plt.show()

In [ ]:
data_test['playersJoined'] = data_test.groupby('matchId')['matchId'].transform('count')

In [ ]:
data_train['party'] = data_train.groupby('groupId')['groupId'].transform('count')
data = data_train.copy()
data = data[data['party']<17]
plt.figure(figsize=(15,10))
sns.countplot(data['party'])
plt.title("party",fontsize=15)
plt.show()

In [ ]:
data_train = data_train[data_train['party']<5]

In [ ]:
data_test['party'] = data_test.groupby('groupId')['groupId'].transform('count')

In [ ]:
data_train['headshot_rate'] = data_train['headshotKills'] / data_train['kills']
data_test['headshot_rate'] = data_test['headshotKills'] / data_test['kills']

In [ ]:
y_train=data_train['winPlacePerc']
id_test = data_test['Id']

In [ ]:
del data_train['Id']
del data_train['groupId']
del data_train['matchId']
del data_train['headshotKills']
del data_train['winPlacePerc']
del data_train['matchType']
del data_test['Id']
del data_test['groupId']
del data_test['matchId']
del data_test['headshotKills']
del data_test['matchType']

In [ ]:
data_train['headshot_rate']=data_train['headshot_rate'].fillna(0)
data_test['headshot_rate']=data_test['headshot_rate'].fillna(0)

In [ ]:
def train(normalize,fit_intercept,label):
    LR = LinearRegression(normalize=normalize,fit_intercept=fit_intercept)
    lr_model = LR.fit(train_x, train_y)
    y_train_predicted = lr_model.predict(train_x)
    y_test_predicted = lr_model.predict(test_x)

    print(f'{label} train {metrics.mean_squared_error(y_train_predicted,train_y)}')
    print(f'{label} test {metrics.mean_squared_error(y_test_predicted,test_y)}')

In [ ]:
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

train_x, test_x, train_y, test_y = train_test_split(data_train,y_train, test_size=0.2)

train(False,False,"nointercept, nonormalize")
train(False,True,"nointercept, normalize")
train(True,False,"intercept, nonormalize")
train(True,True,"intercept, normalize")

In [ ]:
LR = LinearRegression(normalize=True,fit_intercept=False)
lr_model = LR.fit(train_x, train_y)

In [ ]:
probabilities = lr_model.predict(data_test)
pd.DataFrame({
    'Id': id_test,
    'winPlacePerc':     probabilities
}).to_csv('./submission.csv',index=False)